# (E8) Movielens 영화 추천 실습

일시: 2021. 02. 02

## MF 모델 학습 방법으로 내가 좋아할만한 영화 추천 시스템을 제작해보자!

> **추천 시스템이란?**    
나와 비슷한 다른사용자들이 좋아하는 것과 비슷한 것을 추천해 주는 것.    

> **협업 필터링 방식**    
다수의 사용자가 아이템을 구매한 이력정보만으로 사용자 간 유사성 및 아이템 간 유사성을 파악.    
아이템과 사용자 간의 행동/관계에 주목하고 아이템의 고유속성은 주목 X.

> **콘텐츠 기반 필터링**    
아이템 고유의 정보를 바탕으로 아이템 간 유사성을 파악.    
아이템과 사용자 간의 괸계에 주목 X -> 개인화된 추천 제공이 어려움.

*****

# Step 1. 데이터 준비와 전처리

> **데이터는 ```MovieLens 1M Dataset```을 사용.     
이번 프로젝트에서는 별점을 시청 횟수로 해석하고, 유저가 별점을 3점 미만으로 준 데이터는 선호하지 않는다고 가정하고 제외함.**

In [1]:
# rating.dat 안에는 인덱싱이 완료된 사용자-영화-평점 데이터가 정리되어 있음.
import pandas as pd
import os

rating_file_path=os.getenv('HOME') + '/aiffel/recommendata_iu/data/ml-1m/ratings.dat'
ratings_cols = ['user_id', 'movie_id', 'rating', 'timestamp']
ratings = pd.read_csv(rating_file_path, sep='::', names=ratings_cols, engine='python')
orginal_data_size = len(ratings)

ratings.head()

,user_id,movie_id,rating,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [2]:
# 3점 이상만 남김.
ratings = ratings[ratings['rating']>=3]
filtered_data_size = len(ratings)

print(f'orginal_data_size: {orginal_data_size}, filtered_data_size: {filtered_data_size}')
print(f'Ratio of Remaining Data is {filtered_data_size / orginal_data_size:.2%}')

orginal_data_size: 1000209, filtered_data_size: 836478
Ratio of Remaining Data is 83.63%


In [3]:
# rating 컬럼의 이름을 시청횟수로 보기위해 count로 바꿈.
ratings.rename(columns={'rating':'count'}, inplace=True)
ratings['count']

0          5
1          3
2          3
3          4
4          5
          ..
1000203    3
1000205    5
1000206    5
1000207    4
1000208    4
Name: count, Length: 836478, dtype: int64

In [4]:
# 영화 제목을 보기 위해 메타 데이터를 읽어옴.
movie_file_path=os.getenv('HOME') + '/aiffel/recommendata_iu/data/ml-1m/movies.dat'
cols = ['movie_id', 'title', 'genre'] 
movies = pd.read_csv(movie_file_path, sep='::', names=cols, engine='python', encoding = 'latin1')

movies.head()

,movie_id,title,genre
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


In [5]:
# 프레임을 통합해서 보는 것이 처리가 수월할 것 같아 병합.
all = pd.merge(ratings, movies, how = 'inner', on = None)
changed_all = all.drop(['timestamp'], axis = 1) # 필요없는 열 삭제.
changed_all

,user_id,movie_id,count,title,genre
0,1,1193,5,One Flew Over the Cuckoo's Nest (1975),Drama
1,2,1193,5,One Flew Over the Cuckoo's Nest (1975),Drama
2,12,1193,4,One Flew Over the Cuckoo's Nest (1975),Drama
3,15,1193,4,One Flew Over the Cuckoo's Nest (1975),Drama
4,17,1193,5,One Flew Over the Cuckoo's Nest (1975),Drama
...,...,...,...,...,...
836473,5851,3607,5,One Little Indian (1973),Comedy|Drama|Western
836474,5854,3026,4,Slaughterhouse (1987),Horror
836475,5854,690,3,"Promise, The (Versprechen, Das) (1994)",Romance
836476,5938,2909,4,"Five Wives, Three Secretaries and Me (1998)",Documentary


****

# Step 2. 분석하기

In [6]:
print('<ratings에 있는 유니크한 영화 개수> \n', changed_all['title'].nunique()) 
print('<ratings에 있는 유니크한 사용자 수> \n', changed_all['user_id'].nunique()) 
print('<가장 인기있는 영화 30개(인기순)>')
movies_count = all.groupby('title')['user_id'].count() #유저가 향하는 영화 추출.
print(movies_count.sort_values(ascending=False).head(30))

<ratings에 있는 유니크한 영화 개수> 
 3628
<ratings에 있는 유니크한 사용자 수> 
 6039
<가장 인기있는 영화 30개(인기순)>
title
American Beauty (1999)                                   3211
Star Wars: Episode IV - A New Hope (1977)                2910
Star Wars: Episode V - The Empire Strikes Back (1980)    2885
Star Wars: Episode VI - Return of the Jedi (1983)        2716
Saving Private Ryan (1998)                               2561
Terminator 2: Judgment Day (1991)                        2509
Silence of the Lambs, The (1991)                         2498
Raiders of the Lost Ark (1981)                           2473
Back to the Future (1985)                                2460
Matrix, The (1999)                                       2434
Jurassic Park (1993)                                     2413
Sixth Sense, The (1999)                                  2385
Fargo (1996)                                             2371
Braveheart (1995)                                        2314
Men in Black (1997)                     

> **정말 유명한 영화들만 있는 것보니 맞게 나온 것 같음.     
특히 아메리칸 뷰티는 총 89회의 수상과 160회 후보에 지명된 성공적인 영화.**

*****

# Step 3. 내가 선호하는 영화 5가지 골라서 rating에 추가하기

In [7]:
# 무슨 영화제목이 있는지 살펴보기.
# 영화가 너무 많으니 제목을 str.contains() 함수를 사용하여 검색해 영화의 존재여부를 파악하겠음.
changed_all[changed_all['title'].str.contains('Halloween')]

,user_id,movie_id,count,title,genre
577561,19,1982,5,Halloween (1978),Horror
577562,33,1982,5,Halloween (1978),Horror
577563,46,1982,5,Halloween (1978),Horror
577564,53,1982,5,Halloween (1978),Horror
577565,58,1982,4,Halloween (1978),Horror
...,...,...,...,...,...
719878,5654,2107,3,Halloween: H20 (1998),Horror|Thriller
719879,5663,2107,3,Halloween: H20 (1998),Horror|Thriller
719880,5675,2107,3,Halloween: H20 (1998),Horror|Thriller
719881,5788,2107,3,Halloween: H20 (1998),Horror|Thriller


In [8]:
# 내가 선호하는 영화 데이터를 추가.
my_favorite = ['Silence of the Lambs, The (1991)' , 'Nightmare on Elm Street, A (1984)' ,'Friday the 13th (1980)' ,'Halloween (1978)' ,'Shining, The (1980)']

# 'Jeongeun'이라는 user_id의 해당 영화들 시청횟수(별점)는 5라고 가정.
my_playlist = pd.DataFrame({'user_id': ['Jeongeun']*5, 'movie_id' : ['593', '1347', '1974', '1982', '1258'],'title': my_favorite, 'count':5, 'genre':'Horror'})

if not changed_all.isin({'user_id':['Jeongeun']})['user_id'].any():  # user_id에 'Jeongeun'이라는 데이터가 없다면,
    changed_all = changed_all.append(my_playlist)                           # 위에 임의로 만든 my_favorite 데이터를 추가해주기. 

changed_all.tail(10)       # 잘 추가되었는지 확인해보기.

,user_id,movie_id,count,title,genre
836473,5851,3607,5,One Little Indian (1973),Comedy|Drama|Western
836474,5854,3026,4,Slaughterhouse (1987),Horror
836475,5854,690,3,"Promise, The (Versprechen, Das) (1994)",Romance
836476,5938,2909,4,"Five Wives, Three Secretaries and Me (1998)",Documentary
836477,5948,1360,5,Identification of a Woman (Identificazione di ...,Drama
0,Jeongeun,593,5,"Silence of the Lambs, The (1991)",Horror
1,Jeongeun,1347,5,"Nightmare on Elm Street, A (1984)",Horror
2,Jeongeun,1974,5,Friday the 13th (1980),Horror
3,Jeongeun,1982,5,Halloween (1978),Horror
4,Jeongeun,1258,5,"Shining, The (1980)",Horror


In [9]:
# 고유한 유저, 영화를 찾아내는 코드
user_unique = changed_all['user_id'].unique()
title_unique = changed_all['title'].unique()

user_to_idx = {v:k for k,v in enumerate(user_unique)}
title_to_idx = {v:k for k,v in enumerate(title_unique)}

In [10]:
# 인덱싱이 잘 되었는지 확인. 
print(user_to_idx['Jeongeun']) # O.K.   
print(title_to_idx['Silence of the Lambs, The (1991)']) # O.K

6039
121


In [11]:
# indexing을 통해 데이터 컬럼 내 값을 바꾸는 코드

# get(x) 함수: x라는 Key에 대응되는 Value를 반환. 
# map() 함수는 반복가능한 iterable객체를 받아서, 각 요소에 함수를 적용.

# user_to_idx.get을 통해 user_id 컬럼의 모든 값을 인덱싱한 Series를 구하기. 
# 혹시 정상적으로 인덱싱되지 않은 row가 있다면 인덱스가 NaN이 될 테니 dropna()로 제거. 
temp_user_data = changed_all['user_id'].map(user_to_idx.get).dropna()
if len(temp_user_data) == len(changed_all):   # 모든 row가 정상적으로 인덱싱되었다면,
    print('user_id column indexing OK!!')
    changed_all['user_id'] = temp_user_data   # data['user_id']을 인덱싱된 Series로 교체. 
else:
    print('user_id column indexing Fail!!')

# title_to_idx을 통해 title 컬럼도 동일한 방식으로 인덱싱해 줍니다. 
temp_title_data = changed_all['title'].map(title_to_idx.get).dropna()
if len(temp_title_data) == len(changed_all):
    print('title column indexing OK!!')
    changed_all['title'] = temp_title_data
else:
    print('title column indexing Fail!!')

changed_all

user_id column indexing OK!!
title column indexing OK!!


,user_id,movie_id,count,title,genre
0,0,1193,5,0,Drama
1,1,1193,5,0,Drama
2,2,1193,4,0,Drama
3,3,1193,4,0,Drama
4,4,1193,5,0,Drama
...,...,...,...,...,...
0,6039,593,5,121,Horror
1,6039,1347,5,1596,Horror
2,6039,1974,5,1015,Horror
3,6039,1982,5,1016,Horror


*****

> |      |   |   |
|------|---|---|
|명시적|좋아요, 별점 등|
|암묵적|플레이 횟수, 시청 횟수 등|

**--> 시청 횟수가 5에 가까울수록 해당 영화를 더 좋아한다고 판단.**

*****

# Step 4. CSR matrix 구현하기

In [12]:
from scipy.sparse import csr_matrix

num_user = changed_all['user_id'].nunique()
num_title = changed_all['title'].nunique()
csr_data = csr_matrix((changed_all['count'], (changed_all.user_id, changed_all.title)), shape = (num_user, num_title))
csr_data

<6040x3628 sparse matrix of type '<class 'numpy.int64'>'
	with 836483 stored elements in Compressed Sparse Row format>

*****

# Step 5. MF 모델 구성 및 훈련하기

In [13]:
# implicit 패키지: 암묵적인 데이셋을 사용하는 다양한 모델을 빠르게 학습할 수 있음.
# als(AlternatingLeastSquares) 모델을 사용할 것.
# 한쪽을 고정시키고 다른 쪽을 학습하는 방식을 번갈아 수행.

from implicit.als import AlternatingLeastSquares
import os
import numpy as np

os.environ['OPENBLAS_NUM_THREADS']='1'
os.environ['KMP_DUPLICATE_LIB_OK']='True'
os.environ['MKL_NUM_THREADS']='1'

In [14]:
# Implicit AlternatingLeastSquares 모델의 선언
# <클래스 __init__ 파라미터>
# factors: 유저와 아이템의 벡터를 몇 차원으로 할 것인지(과적합 요소)
# reguarization: 과적합을 방지하기 위해 정규화 값을 얼마나 쓸 것인지
# use GPU: GPU를 사용할 것인지
# iteration: 데이터를 얼마나 반복해서 학습할 것인지(과적합 요소)
als_model = AlternatingLeastSquares(factors=200, 
                                    regularization=0.01, 
                                    use_gpu=False, 
                                    iterations=20, 
                                    dtype=np.float32)

In [15]:
# als 모델은 input으로 (item X user 꼴의 matrix를 받기 때문에 Transpose 필요.)
csr_data_transpose = csr_data.T
csr_data_transpose

<3628x6040 sparse matrix of type '<class 'numpy.int64'>'
	with 836483 stored elements in Compressed Sparse Column format>

In [16]:
# 모델 훈련
als_model.fit(csr_data_transpose)

  0%|          | 0/20 [00:00<?, ?it/s]

*****

# Step 6. 모델이 예측한 나의 선호도를 파악해보기

> **내가 선호하는 5가지 영화 중 하나와 그 외의 영화 하나를 골라 훈련된 모델이 예측한 나의 선호도를 파악하기.**

In [17]:
# 모델이 나의 벡터와 양들의 침묵 벡터를 어떻게 만들고 있는지?
# 두 벡터를 곱하면 어떤 값이 나오는지?
je, hannibal = user_to_idx['Jeongeun'], title_to_idx['Silence of the Lambs, The (1991)']
je_vector, hannibal_vector = als_model.user_factors[je], als_model.item_factors[hannibal]

In [18]:
# 사용자 특성 벡터.
je_vector

array([-2.24876314e-01,  2.52989829e-01,  4.51686680e-01,  1.18981443e-01,
       -5.38300335e-01,  2.28627577e-01,  1.49531960e-02,  4.60380793e-01,
       -4.23153728e-01, -8.40134472e-02,  9.40670744e-02,  2.73073651e-02,
        2.69394249e-01, -4.24735188e-01,  4.20425922e-01, -5.98962367e-01,
        1.07404292e+00, -5.02606094e-01,  2.28568017e-01,  9.52712372e-02,
       -7.84393072e-01, -2.52932042e-01, -5.98913670e-01,  9.94185656e-02,
       -4.52209897e-02, -1.13811865e-01,  8.36396124e-03, -2.51660049e-01,
       -2.90288895e-01,  1.69362456e-01,  7.45073408e-02, -5.74864209e-01,
       -1.68727592e-01,  6.02653742e-01,  6.42999947e-01,  6.40340924e-01,
       -4.72548008e-01, -2.58442104e-01,  3.62725109e-01, -4.02045362e-02,
       -1.48359969e-01,  4.18911800e-02, -2.71700602e-02, -8.70293453e-02,
       -1.25431478e-01,  2.62761861e-01,  2.63441861e-01, -5.44630848e-02,
       -8.75019208e-02,  3.50106299e-01, -1.13851048e-01,  7.01303124e-01,
       -4.21949863e-01,  

In [19]:
# 영화의 특성 벡터.
hannibal_vector

array([-1.14167854e-02,  3.47935446e-02,  1.59750171e-02,  2.00154111e-02,
       -1.25384554e-02,  1.61368828e-02,  2.07039490e-02,  3.93672623e-02,
        8.76043178e-03,  4.40277951e-03,  2.19393596e-02,  4.88438411e-03,
        1.78304929e-02, -2.12246533e-02, -2.47934717e-03, -1.41159426e-02,
        3.80273275e-02, -8.07626639e-03,  9.37246345e-03,  4.81730551e-02,
       -1.01090316e-02, -1.13326227e-02, -1.33096743e-02, -1.71973296e-02,
        1.72781721e-02,  1.33744841e-02, -1.83969494e-02, -3.00927199e-02,
       -7.37066567e-03, -3.79429897e-03,  5.69224358e-02, -1.11099863e-02,
       -2.45095454e-02,  2.87383664e-02,  2.90545151e-02,  4.21362743e-03,
       -3.08069238e-03, -3.52568254e-02,  2.48408469e-04,  2.65479903e-03,
        2.48706173e-02,  8.11007898e-03,  2.28848886e-02, -1.43425288e-02,
       -1.36599634e-02, -2.51755342e-02,  6.68636523e-03,  1.18027050e-02,
        1.07763391e-02, -2.33492604e-03,  1.07876514e-03,  1.49315251e-02,
       -7.19137723e-03, -

*****

> **양들의 침묵을 제외한 내가 선호하는 영화들의 특성벡터 4개와      
내가 선호하지 않는 장르의 영화들의 특성벡터 5개를 뽑아 비교해보기.**

In [20]:
# 내가 선호하는 영화들.
freddy, jason, michael, jack = title_to_idx['Nightmare on Elm Street, A (1984)'], title_to_idx['Friday the 13th (1980)'], title_to_idx['Halloween (1978)'], title_to_idx['Shining, The (1980)']
freddy_vector, jason_vector, michael_vector, jack_vector = als_model.item_factors[freddy], als_model.item_factors[jason], als_model.item_factors[michael], als_model.item_factors[jack]

In [21]:
# 내가 선호하지 않는 영화들.
documentary, romance, action, drama, noir = title_to_idx['Five Wives, Three Secretaries and Me (1998)'], title_to_idx['Shakespeare in Love (1998)'], title_to_idx['Terminator, The (1984)'], title_to_idx['Cry, the Beloved Country (1995)'], title_to_idx['Godfather: Part II, The (1974)']
documentary_vector, romance_vector, action_vector, drama_vector, noir_vector = als_model.item_factors[documentary], als_model.item_factors[romance], als_model.item_factors[action], als_model.item_factors[drama], als_model.item_factors[noir]

In [22]:
# 내가 선호하는 영화들의 선호도.
print('나의 \'양들의 침묵\' 선호도 : ', np.dot(je_vector, hannibal_vector))
print('나의 \'나이트메어\' 선호도 : ',np.dot(je_vector, freddy_vector)) 
print('나의 \'13일의 금요일\' 선호도 : ',np.dot(je_vector, jason_vector))
print('나의 \'할로윈\' 선호도 : ',np.dot(je_vector, michael_vector))
print('나의 \'더 샤이닝\' 선호도 : ',np.dot(je_vector, jack_vector))

나의 '양들의 침묵' 선호도 :  0.7829621
나의 '나이트메어' 선호도 :  0.48310548
나의 '13일의 금요일' 선호도 :  0.36932218
나의 '할로윈' 선호도 :  0.4918215
나의 '더 샤이닝' 선호도 :  0.65013444


In [23]:
# 내가 선호하지 않는 영화들의 선호도.
print('나의 \'다큐멘터리 영화\' 선호도 : ',np.dot(je_vector, documentary_vector))
print('나의 \'로맨스 영화\' 선호도 : ',np.dot(je_vector, romance_vector))
print('나의 \'액션 영화\' 선호도 : ',np.dot(je_vector, action_vector))
print('나의 \'드라마 영화\' 선호도 : ',np.dot(je_vector, drama_vector))
print('나의 \'느와르 영화\' 선호도 : ',np.dot(je_vector, noir_vector))

나의 '다큐멘터리 영화' 선호도 :  -0.0013576323
나의 '로맨스 영화' 선호도 :  -0.08967075
나의 '액션 영화' 선호도 :  0.109267816
나의 '드라마 영화' 선호도 :  0.01802406
나의 '느와르 영화' 선호도 :  -0.021028342


> **내가 좋아하는 영화는 대부분 Horro 장르이기 때문에 Romance 장르의 영화는 선호도가 떨어짐을 확인할 수 있음.**

*****

# Step 7. 내가 좋아하는 영화와 비슷한 영화 추천 받기

In [24]:
favorite_movie = 'Re-Animator (1985)' # 좀비 영화.
title_id = title_to_idx[favorite_movie]
similar_movie = als_model.similar_items(title_id, N=15)
similar_movie

[(1681, 0.9999999),
 (1399, 0.68574643),
 (2170, 0.6345794),
 (2148, 0.6276704),
 (883, 0.620286),
 (2084, 0.60858554),
 (1782, 0.6046609),
 (1390, 0.5987861),
 (889, 0.5918769),
 (2843, 0.5887726),
 (1404, 0.587454),
 (990, 0.584716),
 (1680, 0.58236605),
 (1100, 0.567998),
 (1596, 0.56296474)]

In [25]:
# title_to_idx 를 뒤집어, index로부터 영화제목을 얻는 dict를 생성. 
idx_to_title = {v:k for k,v in title_to_idx.items()}
[idx_to_title[i[0]] for i in similar_movie]

['Re-Animator (1985)',
 'Howling, The (1980)',
 'Fog, The (1980)',
 'Bride of Re-Animator (1990)',
 'Phantasm (1979)',
 'Phantasm II (1988)',
 'Fright Night (1985)',
 'Near Dark (1987)',
 'Hidden, The (1987)',
 'Believers, The (1987)',
 'Texas Chainsaw Massacre, The (1974)',
 'American Werewolf in London, An (1981)',
 'Cemetery Man (Dellamorte Dellamore) (1994)',
 'House (1986)',
 'Nightmare on Elm Street, A (1984)']

In [26]:
# 비슷한 영화를 찾아오는 함수로 만들기
def get_similar_movie(title_name: str):
    title_id = title_to_idx[title_name]
    similar_movie = als_model.similar_items(title_id)
    similar_movie = [idx_to_title[i[0]] for i in similar_movie]
    return similar_movie

In [27]:
get_similar_movie('Cemetery Man (Dellamorte Dellamore) (1994)')

['Cemetery Man (Dellamorte Dellamore) (1994)',
 'Two Thousand Maniacs! (1964)',
 'Castle Freak (1995)',
 'Braindead (1992)',
 'Night of the Creeps (1986)',
 'Fury, The (1978)',
 'Hello Mary Lou: Prom Night II (1987)',
 'Bride of Re-Animator (1990)',
 'Addiction, The (1995)',
 'Burnt Offerings (1976)']

*****

# Step 8. 내가 가장 좋아할 만한 영화들 추천 받기

In [28]:
user = user_to_idx['Jeongeun']

# recommend에서는 user*item CSR Matrix를 받음.
movie_recommended = als_model.recommend(user, csr_data, N=20, filter_already_liked_items=True)
movie_recommended

[(374, 0.43771115),
 (1022, 0.34361854),
 (990, 0.33906084),
 (1007, 0.3237748),
 (316, 0.2980287),
 (913, 0.29191294),
 (534, 0.28861868),
 (997, 0.26789856),
 (1028, 0.26455316),
 (51, 0.25265038),
 (279, 0.22879031),
 (1714, 0.22860396),
 (1782, 0.21641704),
 (1358, 0.21601106),
 (726, 0.21339615),
 (157, 0.21151893),
 (197, 0.20425157),
 (247, 0.20389494),
 (257, 0.19703986),
 (1141, 0.19698623)]

In [29]:
[idx_to_title[i[0]] for i in movie_recommended]

['Exorcist, The (1973)',
 'Poltergeist (1982)',
 'American Werewolf in London, An (1981)',
 'Omen, The (1976)',
 'Misery (1990)',
 'Fly, The (1986)',
 'Carrie (1976)',
 'Scream (1996)',
 'Psycho (1960)',
 'Fargo (1996)',
 "Jacob's Ladder (1990)",
 'Christine (1983)',
 'Fright Night (1985)',
 'Birds, The (1963)',
 'Blair Witch Project, The (1999)',
 'Shawshank Redemption, The (1994)',
 'Jaws (1975)',
 'Creepshow (1982)',
 'Night of the Living Dead (1968)',
 'Dead Zone, The (1983)']

> **왜 엑소시스트를 추천해줬을까? 어떤 영화들이 이 추천에 영향을 주었나?**

In [30]:
another = title_to_idx['Exorcist, The (1973)']
explain = als_model.explain(user, csr_data, itemid=another)

In [31]:
# 추천한 콘텐츠의 점수에 기여한 다른 콘텐츠와 기여도(합이 콘텐츠의 점수)를 반환.
[(idx_to_title[i[0]], i[1]) for i in explain[1]] 

[('Shining, The (1980)', 0.19885064551368292),
 ('Halloween (1978)', 0.16460009669893208),
 ('Nightmare on Elm Street, A (1984)', 0.057775806759432585),
 ('Silence of the Lambs, The (1991)', 0.006229317776487897),
 ('Friday the 13th (1980)', 0.003893823659122113)]

> **기여한 영화가 5개 밖에 나오지 않는 이유는 내 유저 아이디로 저장된 영화가 5편 밖에 없기 때문.**

*****

# 루브릭 평가

|평가문항|상세기준|
|:------|:------|
|1. CSR matrix가 정상적으로 만들어졌다.|사용자와 아이템 개수를 바탕으로 정확한 사이즈로 만들었다.|
|2. MF 모델이 정상적으로 훈련되어 그럴듯한 추천이 이루어졌다.|사용자와 아이템 벡터 내적수치가 의미있게 형성되었다.|
|3. 비슷한 영화 찾기와 유저에게 추천하기의 과정이 정상적으로 진행되었다.|MF모델이 예측한 유저 선호도 및 아이템간 유사도, 기여도가 의미있게 측정되었다.|

> **1. changed_all['count']라는 시청횟수 데이터를 changed_all.user_id, changed_all.title이라는 유저와 영화 개수를 바탕으로 정확한 사이즈의 CSR Matrix를 만들었다.**

> **2. 하이퍼 파라미터인 factors와 iterations가 처음에는 100과 15로 설정되어 있었다. 이때는 나의 벡터와 영화의 벡터의 내적(선호도)이 0.5 미만이었다. 그래서 위 두 파라미터를 각각 200과 20으로 바꾸어서 양들의 침묵의 경우에는 선호도가 0.7 이상을 넘길 수 있었다.**    
>
> **선호하는 영화와 선호하지 않는 영화들의 선호도를 비교해보면 그 차이를 더욱 확연하게 알 수 있다. 선호하는 영화의 경우, 선호도가 0.5 주변에 있거나 그 이상이지만 선호하지 않는 영화의 경우에는 0 주변에 있고 몇몇은 음수임을 알 수 있다.**
>
> **1에 가까울수록 선호도가 높은 것으로 간주하기로 했는데, 13일의 금요일과 할로윈이 각각 0.4와 0.3 대로 낮은 선호도를 표시하고 있어서 약간 의외라고 생각하고 있다.**

> **3.나는 선호하는 영화를 모두 호러 영화로 골랐다. 추천받은 비슷한 영화들도 대부분 호러 영화인 것으로 확인되었다. 따라서 선호도는 의미있게 측정된 것으로 보인다.**
>
> **내가 추천받은 영화에 대한 기여도는 생각보다 낮게 나왔다. 비슷한 장르인 것을 감안하면 의외인 결과였다. 추측하건데, 내가 추천받은 영화는 '엑소시스트'였기에 호러 장르지만 동시에 귀신이 나오는 영화였다.     
하지만 내가 선호한다고 말한 영화는 살인마가 나오는 슬래셔 무비나 스릴러에 가까운 호러 영화였기에 상대적으로 주인공으로 내새우는 인물이 달라 기여도가 낮게 나온 것 같다.** 

*****

# 회고

 **이번 프로젝트를 통해서 도메인이 얼마나 중요한지 다시금 깨달았다. 나는 영화를 좋아하는터라 이번 도메인이 영화라서 정말 재밌었다. 결과가 제대로 나왔는지 아닌지 내가 보고 판단할 수 있으니 프로젝트를 진행하는 것이 한결 수월했다.**     

 **CSR Matrix를 처음 접했을 땐 정말 어려웠다. 그래도 손으로 행렬을 그리면서 하니 이해가 됐다. 0과 같은 빈 공간을 절약해서 필요한 정보만 넣는다니 얼마나 효율적인지! 이런걸 직접 생각해 낸 사람도 신기하고, 그 방법을 추천시스템에 적용할 생각을 했다는 것도 놀라웠다.**
 
 **내가 Flask 같은 것을 사용할 줄 알았다면, 영화를 입력해서 비슷한 영화를 추천해주는 웹 페이지(?)같은 것을 만들어봐도 좋을 거란 생각이 들었다. 하지만 항상 걸리는 것은 나의 코딩 실력이다.**
 
**그래도 이번 프로젝트는 정말 재밌었고,정말 왓챠나 넷플릭스에서 볼법한 실무적인 프로젝트를 해낸 것 같아 마음이 참 뿌듯하다.**